In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
history_df = pd.DataFrame()
file_path = "history.csv"
if os.path.isfile(file_path) and os.path.getsize(file_path) > 0:
    history_df = pd.read_csv(file_path)
else:
    print("Error: The file is empty or does not exist.")

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_68860/4205743744.py:4: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  history_df = pd.read_csv(file_path)


In [4]:
'''
@param df: A pandas DataFrame
@param cols: List of columns existing in df
@return: df with cols dropped
'''
def drop_cols(df, cols):
    df = df.drop(cols, axis=1)
    return df

In [5]:
'''
@param df: A pandas DataFrame
@param cols: List of boolean columns in df
@return: df with the values in cols mapped to 0 (for False or blank) or 1 (for True)
'''
def boolean_map(df, cols):
    for col in cols:
        df[col] = df[col].fillna('f').map({'t': 1, 'f': 0})
    return df

In [6]:
'''
@param df: A pandas DataFrame
@param cols: List of datetime columns in df
@return: df with the values in cols standardized to a number between 0 and 1.
'''
def parse_time(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col])
        min_date = df[col].min()
        df[col] = df[col].fillna(min_date)

        df[col] = df[col].astype(int)
        min = df[col].min()
        max = df[col].max()    
        df[col] = (df[col] - min) / (max - min)

    return df

In [7]:
'''
@param df: A pandas DataFrame
@param cols: List of object columns in df
@return: df with the values in cols mapped to arbitrary numbers
'''
def encode_labels(df, cols):
    label_encoder = LabelEncoder()
    for col in cols:
        df[col] = label_encoder.fit_transform(df[col])
    
    return df


### 1.0 Drop unused columns

We do not want our model to be trained on data from any of these columns. The code looks a bit messy because we did this step in different ways.

In [9]:
df_nathan = history_df[['id', 'type_code', 'class_id', 'inception_date', 'branch', 'include_client_consolidation', 'use_client_address', 'credit_limit_type', 'is_arp_locked', 'dividend_confirm_code']]

df_cloris = history_df[['options_trading_type', 'interest_dividend_conversion_type', 'guarantee_gtor_type', 'share_name_address_to_issuer', 'shareholder_instructions_received', 'is_tms_eligible', 'is_agent_bbs_participant', 'is_spousal_transfer', 'terminal_code', 'deceased_fair_market_value', 'discretionary_trading_authorized']]

df_cherry = history_df.iloc[:, 53:78]
df_cherry = drop_cols(df_cherry, ['esir_number', 'dup_trip_quad_code', 'function_code', 'portfolio_cost_method', 'interactive_portfolio_code'])

df_matthew = history_df.iloc[:, -27:]
df_matthew = drop_cols(df_matthew, ['is_broker_account', 'is_gl_account', 'is_control_account', 'is_plan_grandfathered'])

training_df = pd.concat([df_nathan, df_cloris, df_cherry, df_matthew], axis=1)
training_df.head(3)

,id,type_code,class_id,inception_date,branch,include_client_consolidation,use_client_address,credit_limit_type,is_arp_locked,dividend_confirm_code,...,is_family_resp,is_hrdc_resp,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,893724,CASH SWEEP,3.0,2017-08-18,IAVM,t,f,0.0,NaN,0.0,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,f,Churn
1,1268094,RRIF,5.0,2018-11-08,IAVM,t,t,0.0,f,0.0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2023-01-03 00:00:00,BATCH,t,Churn
2,606613,SPOUSAL RRSP,5.0,2009-04-08,IAVM,f,f,0.0,f,0.0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2018-07-17 00:00:00,GUERINO,f,Churn


### 1.1 Map Boolean Columns

In [10]:
bool_nathan = ['include_client_consolidation', 'use_client_address', 'is_arp_locked']
bool_cloris = ['share_name_address_to_issuer', 'shareholder_instructions_received', 'is_tms_eligible', 'is_agent_bbs_participant', 'is_spousal_transfer', 'discretionary_trading_authorized']
bool_cherry = ['receive_general_mailings', 'has_discrete_auth', 'is_non_objecting_beneficial_owner', 'is_objecting_to_disclose_info', 'consent_to_email_delivery', 'consent_to_pay_for_mail', 'has_received_instruction']
bool_matthew = ['is_inventory_account', 'is_extract_eligible', 'is_pledged', 'is_resp', 'is_family_resp', 'is_hrdc_resp'] + ['is_olob', 'visible_in_reports', 'use_original_date_for_payment_calc']

training_df = boolean_map(training_df, bool_nathan + bool_cloris + bool_cherry + bool_matthew)

### 1.2 Parse Datetime Columns

In [11]:
time_nathan = ['inception_date']
time_cherry = ['last_update_date']
time_matthew = ['last_maintenance_time', 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time']

training_df = parse_time(training_df, time_nathan + time_cherry + time_matthew)

### 1.3 Numerize Type Columns

In [13]:
type_nathan = ['type_code', 'branch']
type_matthew = ['last_maintenance_user', 'retail_last_maintenance_user']

training_df = encode_labels(training_df, type_nathan + type_matthew)

### 1.4 Other

In [14]:
# Cloris
training_df.deceased_fair_market_value.fillna(0, inplace=True)

# Matthew
training_df.number_of_beneficiaries.fillna(0, inplace=True)
training_df.label = training_df.label.map({'No Churn': 0, 'Churn': 1})

In [15]:
training_df.to_csv("training_df.csv")
training_df

,id,type_code,class_id,inception_date,branch,include_client_consolidation,use_client_address,credit_limit_type,is_arp_locked,dividend_confirm_code,...,is_family_resp,is_hrdc_resp,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,893724,1,3.0,0.808752,5,1,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.835465,0,0.000000,432,0,1
1,1268094,15,5.0,0.847716,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.954733,25,1,1
2,606613,18,5.0,0.542538,5,0,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.633734,174,0,1
3,741930,0,3.0,0.916405,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.835465,0,0.000000,432,1,1
4,1137922,0,3.0,0.806311,5,1,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.836450,0,0.000000,432,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673334,6827067,8,5.0,0.995206,2,1,1,0.0,1,0.0,...,0,0,NaN,0.34237,0.861687,0,0.989372,25,1,1
673335,590659,13,5.0,0.466963,5,1,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.626845,25,0,0
673336,1247774,0,3.0,0.873431,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.835465,0,0.000000,432,1,1
673337,1155640,15,5.0,0.806311,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.972643,374,1,1


Unprocessed columns



Matthew
- special_fee_code: numbers, characters, blanks
- resp_specimen plan: numbers (like ids) and blanks